In [ ]:
from zipfile import ZipFile
# from zipfile import BadZipFile
# specifying the zip file name
file_name = "/content/Brain_2.zip"
  
# opening the zip file in READ mode
with ZipFile(file_name, 'r') as zip:
    # printing all the contents of the zip file
    zip.printdir()
    # extracting all the files
    print('Extracting all the files now...')
    zip.extractall()
    print('Done!')

File Name                                             Modified             Size
Brain_2/                                       2023-04-17 18:49:24            0
Brain_2/Test/                                  2023-04-17 18:52:34            0
Brain_2/Test/No tumor/                         2023-04-17 18:52:12            0
Brain_2/Test/No tumor/Test-notumor/            2023-04-17 18:47:24            0
Brain_2/Test/No tumor/Test-notumor/no501.jpg   2021-11-14 04:39:44        12392
Brain_2/Test/No tumor/Test-notumor/no502.jpg   2021-11-14 04:39:44        11695
Brain_2/Test/No tumor/Test-notumor/no503.jpg   2021-11-14 04:39:44        42451
Brain_2/Test/No tumor/Test-notumor/no504.jpg   2021-11-14 04:39:44        14360
Brain_2/Test/No tumor/Test-notumor/no505.jpg   2021-11-14 04:39:44       127722
Brain_2/Test/No tumor/Test-notumor/no506.jpg   2021-11-14 04:39:44        18154
Brain_2/Test/No tumor/Test-notumor/no507.jpg   2021-11-14 04:39:44        11761
Brain_2/Test/No tumor/Test-notumor/no508

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

# Define data generator with augmentation for training set
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

# Define data generator without augmentation for validation and test sets
val_test_datagen = ImageDataGenerator(rescale=1./255)

# Specify directories for training, validation, and test sets
train_directory = '/content/Brain_2/Train'  # Directory containing training images
val_directory = '/content/Brain_2/Train'  # Directory containing validation images
test_directory = '/content/Brain_2/Test'  # Directory containing test images

# Specify target size, batch size, class mode, and color mode
target_size = (224, 224)
batch_size = 32
class_mode = 'binary'
shuffle = True
seed = 42
color_mode = 'rgb'

# Generate batches of data for training set
train_generator = train_datagen.flow_from_directory(
    train_directory,
    target_size=target_size,
    batch_size=batch_size,
    class_mode=class_mode,
    subset='training'
    # shuffle=shuffle,
    # seed=seed,
    # color_mode=color_mode
)

# Generate batches of data for validation set
val_generator = val_test_datagen.flow_from_directory(
    val_directory,
    target_size=target_size,
    batch_size=batch_size,
    class_mode=class_mode,
    subset='validation'
    # shuffle=shuffle,
    # seed=seed,
    # color_mode=color_mode
)

# Generate batches of data for test set
test_generator = val_test_datagen.flow_from_directory(
    test_directory,
    target_size=target_size,
    batch_size=batch_size,
    class_mode=class_mode,
    # shuffle=shuffle,
    # seed=seed,
    # color_mode=color_mode
)

# Now, you can use `train_generator`, `val_generator`, and `test_generator`
# with the appropriate functions for training, validation, and testing your model.

from keras.utils import to_categorical





Found 1001 images belonging to 2 classes.
Found 0 images belonging to 2 classes.
Found 200 images belonging to 2 classes.


Resnet50 model


In [ ]:
from keras.applications import ResNet50
from keras.layers import Dense, GlobalAveragePooling2D, Flatten
from keras.models import Sequential
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
import numpy as np

# Load the ResNet50 model
base_model = ResNet50(include_top=False, weights='imagenet', input_shape=(target_size[0], target_size[1], 3))

# Create a Sequential model
model = Sequential()

# Add the base ResNet50 model to the Sequential model
model.add(base_model)

# Add top layers for classification
# model.add(GlobalAveragePooling2D())
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer=Adam(lr=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit_generator(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=10,
    validation_data=val_generator,
    validation_steps=val_generator.samples // batch_size
)

# Evaluate the model on the test set
score = model.evaluate_generator(
    test_generator,
    steps=test_generator.samples // batch_size
)

print('Test loss:', score[0])
print('Test accuracy:', score[1])
# Assuming you have a trained sequential ResNet model called `model` and a test generator called `test_generator`



<ipython-input-45-3fafd1b74b60>:28: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(


Epoch 1/10
31/31 [==============================] - 21s 483ms/step - loss: 0.3911 - accuracy: 0.8493
Epoch 2/10
31/31 [==============================] - 15s 485ms/step - loss: 0.3100 - accuracy: 0.8875
Epoch 3/10
31/31 [==============================] - 15s 484ms/step - loss: 0.2355 - accuracy: 0.9051
Epoch 4/10
31/31 [==============================] - 15s 484ms/step - loss: 0.1662 - accuracy: 0.9401
Epoch 5/10
31/31 [==============================] - 15s 479ms/step - loss: 0.1412 - accuracy: 0.9536
Epoch 6/10
31/31 [==============================] - 15s 477ms/step - loss: 0.1089 - accuracy: 0.9670
Epoch 7/10
31/31 [==============================] - 15s 477ms/step - loss: 0.1076 - accuracy: 0.9546
Epoch 8/10
31/31 [==============================] - 16s 510ms/step - loss: 0.1182 - accuracy: 0.9556
Epoch 9/10
31/31 [==============================] - 15s 484ms/step - loss: 0.0901 - accuracy: 0.9649
Epoch 10/10
31/31 [==============================] - 15s 493ms/step - loss: 0.0770 - accura

<ipython-input-45-3fafd1b74b60>:37: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  score = model.evaluate_generator(


Test loss: 0.7295450568199158
Test accuracy: 0.5052083134651184


Precision

In [ ]:
# Presicion
import numpy as np
# Use the trained model to predict labels for the test data
predicted_probs = model.predict_generator(test_generator)

# Convert predicted probabilities into binary values (0 or 1) based on a threshold
threshold = 0.5
predicted_labels_binary = (predicted_probs > threshold).astype(int)

# Get true labels of the test data from the test generator
true_labels = test_generator.classes

# Calculate true positives (TP), false positives (FP), true negatives (TN), and false negatives (FN)
TP = np.sum((predicted_labels_binary == 1) & (true_labels == 1))
FP = np.sum((predicted_labels_binary == 1) & (true_labels == 0))
TN = np.sum((predicted_labels_binary == 0) & (true_labels == 0))
FN = np.sum((predicted_labels_binary == 0) & (true_labels == 1))

# Calculate precision
precision = TP / (TP + FP)
print('precision : ',precision)

<ipython-input-47-6bd74b55ff2e>:3: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  predicted_probs = model.predict_generator(test_generator)


precision :  0.5


Resnet50 with 50 epochs

In [ ]:
# from keras.applications.resnet50 import ResNet50
from keras.applications import ResNet50
from keras.layers import Dense, GlobalAveragePooling2D , Flatten
from keras.models import Model
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
base_model = ResNet50(include_top=False, weights='imagenet', input_shape=(target_size[0], target_size[1], 3))

# Add top layers for classification
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Flatten()(x)  
x = Dense(256, activation='relu')(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)  # Binary classification, so sigmoid activation for the output layer

# Create the final model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer=Adam(lr=0.001), loss='binary_crossentropy', metrics=['accuracy'])



# Train the model
# model.fit_generator(
#     train_generator,
#     steps_per_epoch=train_generator.samples // batch_size,
#     epochs=20,  # Example number of epochs
#     validation_data=val_generator,
#     validation_steps=val_generator.samples // batch_size
# )

model.fit_generator(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=50,
    validation_data=val_generator,
    validation_steps=val_generator.samples // batch_size,
    # y=train_labels  # Pass train_labels as y parameter
)

# Evaluate the model on the test set
score = model.evaluate_generator(
    test_generator,
    steps=test_generator.samples // batch_size
)

print('Test loss:', score[0])
print('Test accuracy:', score[1])

<ipython-input-17-bcfddc2e811d>:34: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(


Epoch 1/50
31/31 [==============================] - 21s 512ms/step - loss: 0.3265 - accuracy: 0.8751
Epoch 2/50
31/31 [==============================] - 16s 512ms/step - loss: 0.1808 - accuracy: 0.9381
Epoch 3/50
31/31 [==============================] - 16s 511ms/step - loss: 0.1545 - accuracy: 0.9536
Epoch 4/50
31/31 [==============================] - 15s 490ms/step - loss: 0.1363 - accuracy: 0.9474
Epoch 5/50
31/31 [==============================] - 16s 511ms/step - loss: 0.0748 - accuracy: 0.9773
Epoch 6/50
31/31 [==============================] - 15s 488ms/step - loss: 0.1463 - accuracy: 0.9515
Epoch 7/50
31/31 [==============================] - 15s 490ms/step - loss: 0.1477 - accuracy: 0.9432
Epoch 8/50
31/31 [==============================] - 15s 501ms/step - loss: 0.0605 - accuracy: 0.9804
Epoch 9/50
31/31 [==============================] - 15s 483ms/step - loss: 0.1240 - accuracy: 0.9536
Epoch 10/50
31/31 [==============================] - 16s 505ms/step - loss: 0.0753 - accura

<ipython-input-17-bcfddc2e811d>:44: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  score = model.evaluate_generator(


Test loss: 2.3943254947662354
Test accuracy: 0.8229166865348816


In [ ]:
from keras.applications import ResNet50
from keras.layers import Dropout, Flatten, Dense
from keras.models import Sequential
from keras.optimizers import RMSprop, Adam

NUM_CLASSES = 1

# Load pre-trained ResNet50 model
resnet50 = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

model = Sequential()
model.add(resnet50)
model.add(Dropout(0.3))
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(NUM_CLASSES, activation='sigmoid'))

model.layers[0].trainable = False

model.compile(
    loss='binary_crossentropy',
    optimizer=RMSprop(lr=1e-4),
    metrics=['accuracy']
)

# Alternatively, you can use Adam optimizer with custom learning rate and other hyperparameters
model.compile(
    loss='binary_crossentropy',
    optimizer=Adam(lr=0.0003, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False),
    metrics=["accuracy"]
)

model.summary()


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 dropout_1 (Dropout)         (None, 7, 7, 2048)        0         
                                                                 
 flatten_7 (Flatten)         (None, 100352)            0         
                                                                 
 dropout_2 (Dropout)         (None, 100352)            0         
                                                                 
 dense_32 (Dense)            (None, 1)                 100353    
                                                                 
Total params: 23,688,065
Trainable params: 100,353
Non-trainable params: 23,587,712
_________________________________________________________________


/usr/local/lib/python3.9/dist-packages/keras/optimizers/legacy/rmsprop.py:143: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [ ]:


# Evaluate model on test dataset
test_loss, test_accuracy = model.evaluate(test_generator, steps=len(test_generator))
print('Test Loss:', test_loss)
print('Test Accuracy:', test_accuracy)


7/7 [==============================] - 3s 189ms/step - loss: 0.7165 - accuracy: 0.4250
Test Loss: 0.716461718082428
Test Accuracy: 0.42500001192092896


Vgg16 model


In [ ]:
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras.optimizers import RMSprop, Adam

# Load the VGG16 model
vgg = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
for layer in vgg.layers:
  layer.trainable=True

# Create a Sequential model
vgg16 = Sequential()

# Add the VGG16 model as the first layer in the Sequential model
vgg16.add(vgg)

# Add additional layers to the Sequential model
vgg16.add(Flatten())
vgg16.add(Dense(512, activation='relu'))
vgg16.add(Dense(256, activation='relu'))
vgg16.add(Dense(256, activation='relu'))
vgg16.add(Dense(1, activation='sigmoid'))

# Compile the model
vgg16.compile(loss='binary_crossentropy',
              optimizer=RMSprop(lr=1e-4),
              metrics=['accuracy'])

# Print the model summary
vgg16.summary()

# Train the model
vgg16.fit(train_generator,
          steps_per_epoch=train_generator.samples // batch_size,
          epochs=20,
          validation_data=val_generator,
          validation_steps=val_generator.samples // batch_size)

# Evaluate the model on the test data
test_loss, test_accuracy = vgg16.evaluate(test_generator,
                                          steps=test_generator.samples // batch_size)
print('Test Loss:', test_loss)
print('Test Accuracy:', test_accuracy)


58889256/58889256 [==============================] - 3s 0us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 25088)             0         
                                                                 
 dense (Dense)               (None, 512)               12845568  
                                                                 
 dense_1 (Dense)             (None, 256)               131328    
                                                                 
 dense_2 (Dense)             (None, 256)               65792     
                                                                 
 dense_3 (Dense)             (None, 1)                 257       
                                                         

/usr/local/lib/python3.9/dist-packages/keras/optimizers/legacy/rmsprop.py:143: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/20
31/31 [==============================] - 41s 642ms/step - loss: 0.6886 - accuracy: 0.5531
Epoch 2/20
31/31 [==============================] - 16s 512ms/step - loss: 0.6584 - accuracy: 0.6749
Epoch 3/20
31/31 [==============================] - 16s 520ms/step - loss: 0.6149 - accuracy: 0.6811
Epoch 4/20
31/31 [==============================] - 17s 523ms/step - loss: 0.5055 - accuracy: 0.7740
Epoch 5/20
31/31 [==============================] - 16s 521ms/step - loss: 0.4331 - accuracy: 0.8050
Epoch 6/20
31/31 [==============================] - 17s 528ms/step - loss: 0.4420 - accuracy: 0.7988
Epoch 7/20
31/31 [==============================] - 17s 548ms/step - loss: 0.3589 - accuracy: 0.8431
Epoch 8/20
31/31 [==============================] - 17s 523ms/step - loss: 0.2851 - accuracy: 0.8865
Epoch 9/20
31/31 [==============================] - 16s 521ms/step - loss: 0.3310 - accuracy: 0.8679
Epoch 10/20
31/31 [==============================] - 17s 527ms/step - loss: 0.2465 - accura

Acuuracy: 94
which is the best 


In [ ]:
import pickle
pickle.dump(vgg16, open('vgg16.pkl', 'wb'))


